In [ ]:
# from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, MinMaxScaler
# from pyspark.ml.clustering import KMeans
# from pyspark.sql import functions as F
# import sys
# from pyspark.sql import SparkSession
# import matplotlib.pyplot as plt
# from pyspark.sql import functions as F
# import seaborn as sns
# from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, BooleanType

In [ ]:
# spark = SparkSession.builder \
#     .appName("Data Processing dans catalogue csv") \
#     .config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore:9083") \
#     .enableHiveSupport() \
#     .getOrCreate()
# print("Session Spark initialisée avec succès.")
# spark.sql("USE concessionnaire")
# catalogue_df = spark.sql("SELECT * FROM catalogue_co2_merge_processed")

In [ ]:
# catalogue_df.show(10)
# catalogue_df.printSchema()


In [ ]:
# from pyspark.sql import SparkSession
# from pyspark.sql import functions as F
# from pyspark.sql.functions import when, col, lit
# from pyspark.ml.feature import VectorAssembler, MinMaxScaler
# from pyspark.ml.clustering import KMeans
# 
# # Initialisation de la session Spark
# spark = SparkSession.builder \
#     .appName("Clustering Véhicules") \
#     .enableHiveSupport() \
#     .getOrCreate()
# print("Session Spark initialisée avec succès.")
# spark.sql("USE concessionnaire")
# 
# # Charger les données
# catalogue_df = spark.sql("SELECT * FROM catalogue_co2_merge_processed")
# print(f"Nombre de lignes dans le DataFrame initial : {catalogue_df.count()}")
# catalogue_df.show(5, truncate=False)
# 
# # Étape 0 : Correction des anomalies dans 'rejets_co2'
# 
# # Identifier les anomalies
# anomalies_df = catalogue_df.filter(
#     (col('rejets_co2').isNull()) |
#     (col('rejets_co2') == 0) |
#     ((col('unified_horse_power') > 100) & (col('rejets_co2') < 50))
# )
# print(f"Nombre d'anomalies détectées : {anomalies_df.count()}")
# 
# # Calcul des moyennes
# avg_co2_by_model = F.broadcast(catalogue_df.filter(col('rejets_co2') > 0)
#                                .groupBy('marque', 'modele')
#                                .agg(F.avg('rejets_co2').alias('avg_rejets_co2_model')))
# 
# avg_co2_by_marque = F.broadcast(catalogue_df.filter(col('rejets_co2') > 0)
#                                 .groupBy('marque')
#                                 .agg(F.avg('rejets_co2').alias('avg_rejets_co2_marque')))
# 
# avg_co2_general = catalogue_df.filter(col('rejets_co2') > 0) \
#     .agg(F.avg('rejets_co2').alias('avg_rejets_co2_general')).collect()[0]['avg_rejets_co2_general']
# 
# # Imputation des anomalies
# anomalies_df = anomalies_df \
#     .join(avg_co2_by_model, on=['marque', 'modele'], how='left') \
#     .join(avg_co2_by_marque, on='marque', how='left') \
#     .withColumn('rejets_co2', when(col('avg_rejets_co2_model').isNotNull(), col('avg_rejets_co2_model'))
#                 .when(col('avg_rejets_co2_marque').isNotNull(), col('avg_rejets_co2_marque'))
#                 .otherwise(lit(avg_co2_general))) \
#     .drop('avg_rejets_co2_model', 'avg_rejets_co2_marque')
# 
# # Mise à jour des données
# catalogue_corrected_df = catalogue_df.join(anomalies_df.select('marque', 'modele', 'rejets_co2'), 
#                                            on=['marque', 'modele'], how='left_anti').unionByName(anomalies_df)
# print("Données corrigées prêtes.")
# catalogue_corrected_df.show(5, truncate=False)
# 
# # Étape 1 : Préparation des données pour le clustering
# numerical_cols = ['prix', 'unified_horse_power', 'rejets_co2', 'bonus_malus', 'nbplaces', 'nbportes']
# clustering_df = catalogue_corrected_df.select(
#     *[col(c).cast('double').alias(c) for c in numerical_cols],
#     'marque', 'modele'
# ).fillna({c: 0 for c in numerical_cols})
# 
# assembler = VectorAssembler(inputCols=numerical_cols, outputCol='features_raw')
# assembled_df = assembler.transform(clustering_df)
# 
# scaler = MinMaxScaler(inputCol='features_raw', outputCol='features')
# scaled_df = scaler.fit(assembled_df).transform(assembled_df)
# print("Données normalisées prêtes pour le clustering.")
# scaled_df.select("features").show(5, truncate=False)
# 
# # Étape 2 : Clustering avec K-Means
# k = 9
# kmeans = KMeans(featuresCol='features', k=k, seed=1)
# model = kmeans.fit(scaled_df)
# clustered_df = model.transform(scaled_df)
# 
# # Étape 3 : Analyse des clusters
# cluster_stats = clustered_df.groupBy('prediction').agg(
#     F.count('*').alias('count'),
#     F.mean('prix').alias('mean_prix'),
#     F.mean('unified_horse_power').alias('mean_horse_power'),
#     F.mean('rejets_co2').alias('mean_rejets_co2'),
#     F.mean('bonus_malus').alias('mean_bonus_malus'),
#     F.mean('nbplaces').alias('mean_nbplaces'),
#     F.mean('nbportes').alias('mean_nbportes')
# ).orderBy('prediction')
# 
# print("Statistiques des clusters :")
# cluster_stats.show(truncate=False)
# 
# # Étape 4 : Assignation des catégories
# clustered_df = clustered_df.withColumn(
#     "categorie",
#     when(col("prediction") == 0, "Berlines hybrides haut de gamme") 
#     .when(col("prediction") == 1, "Berlines de luxe")
#     .when(col("prediction") == 2, "Voitures sportives haut de gamme")
#     .when(col("prediction") == 3, "Citadines économiques")
#     .when(col("prediction") == 4, "Familiales grandes places")
#     .when(col("prediction") == 5, "Berlines intermédiaires thermiques")
#     .when(col("prediction") == 6, "Micro-citadines électriques") 
#     .when(col("prediction") == 7, "Citadines")
#     .when(col("prediction") == 8, "Citadines compactes")
#     .otherwise("Autres")
# )
# 
# # Étape 5 : Vérification des catégories
# categories = clustered_df.select("categorie").distinct().rdd.flatMap(lambda x: x).collect()
# for cat in categories:
#     print(f"\nExemples pour la catégorie : {cat}")
#     clustered_df.filter(col("categorie") == cat).select(
#         "marque", "modele", "prix", "unified_horse_power", "rejets_co2", "nbplaces", "nbportes"
#     ).show(5, truncate=False)


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import when, col, lit
from pyspark.ml.feature import VectorAssembler, MinMaxScaler, PCA
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
import matplotlib.pyplot as plt

# Initialisation de la session Spark
spark = SparkSession.builder \
    .appName("Clustering Véhicules") \
    .config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()
print("Session Spark initialisée avec succès.")
spark.sql("USE concessionnaire")

# Charger les données
catalogue_df = spark.sql("SELECT * FROM catalogue_co2_merge_processed")
print(f"Nombre de lignes dans le DataFrame initial : {catalogue_df.count()}")
catalogue_df.show(5, truncate=False)

# Étape 1 : Préparation des données pour le clustering
numerical_cols = ['prix', 'unified_horse_power', 'rejets_co2', 'bonus_malus', 'nbplaces', 'nbportes']
clustering_df = catalogue_df.select(
    *[col(c).cast('double').alias(c) for c in numerical_cols],
    'marque', 'modele'
).fillna({c: 0 for c in numerical_cols})

assembler = VectorAssembler(inputCols=numerical_cols, outputCol='features_raw')
assembled_df = assembler.transform(clustering_df)

scaler = MinMaxScaler(inputCol='features_raw', outputCol='features')
scaled_df = scaler.fit(assembled_df).transform(assembled_df)

print("Données normalisées prêtes pour le clustering :")
scaled_df.select("features").show(5, truncate=False)

# Étape 2 : Clustering avec K-Means
k = 9
kmeans = KMeans(featuresCol='features', k=k, seed=1)
model = kmeans.fit(scaled_df)
clustered_df = model.transform(scaled_df)

# Calcul du score de silhouette
evaluator = ClusteringEvaluator(featuresCol='features', metricName='silhouette')
silhouette_score = evaluator.evaluate(clustered_df)
print(f"Score de silhouette pour k={k}: {silhouette_score:.4f}")

# Étape 3 : Analyse des clusters
print("\n--- Répartition des clusters ---")
clustered_df.groupBy('prediction').count().orderBy('prediction').show()

print("\n--- Statistiques des clusters ---")
cluster_stats = clustered_df.groupBy('prediction').agg(
    F.count('*').alias('count'),
    *[F.mean(c).alias(f'mean_{c}') for c in numerical_cols],
    *[F.stddev(c).alias(f'stddev_{c}') for c in numerical_cols]
)
cluster_stats.show(truncate=False)

# Étape 4 : Réduction de dimensions pour visualisation
print("\n--- Réduction de dimensions avec PCA ---")
pca = PCA(k=2, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(clustered_df)
pca_df = pca_model.transform(clustered_df)

# Collecte des données pour Matplotlib
pca_data = pca_df.select("pca_features", "prediction").rdd.map(
    lambda row: (row["pca_features"][0], row["pca_features"][1], row["prediction"])
).collect()

x_coords = [x[0] for x in pca_data]
y_coords = [x[1] for x in pca_data]
predictions = [x[2] for x in pca_data]

# Visualisation des clusters
plt.figure(figsize=(10, 6))
scatter = plt.scatter(x_coords, y_coords, c=predictions, cmap="tab10", alpha=0.7)
plt.colorbar(scatter, label="Cluster")
plt.title(f"Visualisation des clusters (PCA) - Silhouette Score: {silhouette_score:.4f}")
plt.xlabel("Composante principale 1")
plt.ylabel("Composante principale 2")
plt.grid(True)
plt.show()

# Étape 5 : Vérification des catégories
print("\n--- Exemples pour chaque cluster ---")
categories = clustered_df.select("prediction").distinct().rdd.flatMap(lambda x: x).collect()
for cluster in categories:
    print(f"\n--- Exemples pour le cluster {cluster} ---")
    clustered_df.filter(col("prediction") == cluster).select(
        "marque", "modele", "prix", "unified_horse_power", "rejets_co2", "nbplaces", "nbportes"
    ).show(5, truncate=False)


In [ ]:
clustered_df.groupBy("prediction").count().show()


In [ ]:
# Ajouter des catégories pour chaque cluster
clustered_df = clustered_df.withColumn(
    "categorie",
    when(col("prediction") == 1, "Berlines compactes économiques")
    .when(col("prediction") == 6, "Compactes premium")
    .when(col("prediction") == 5, "Monospaces familiaux 7 places")
    .when(col("prediction") == 4, "Berlines de luxe haut de gamme")
    .when(col("prediction") == 8, "Voitures sportives haut de gamme")
    .when(col("prediction") == 2, "Citadines économiques compactes")
    .when(col("prediction") == 0, "Berlines routières intermédiaires")
    .when(col("prediction") == 3, "Micro-citadines économiques")
    .when(col("prediction") == 7, "Monospaces compacts économiques")
    .otherwise("Non défini")
)

# Afficher les données avec les catégories
print("\n--- Véhicules avec catégories assignées ---")
clustered_df.select(
    "marque", "modele", "prix", "unified_horse_power", "rejets_co2", "nbplaces", "nbportes", "categorie"
).show(100, truncate=False)

# Sauvegarder les résultats pour un usage ultérieur
# clustered_df.write.mode('overwrite').saveAsTable("concessionnaire.clustered_with_categories")


In [ ]:
# Grouper les résultats par catégorie et afficher des exemples pour chaque
categories = clustered_df.select("categorie").distinct().rdd.flatMap(lambda x: x).collect()

for cat in categories:
    print(f"\n--- Exemples pour la catégorie : {cat} ---")
    clustered_df.filter(col("categorie") == cat).select(
        "marque", "modele", "prix", "unified_horse_power", "rejets_co2", "nbplaces", "nbportes", "categorie"
    ).show(10, truncate=False)


In [ ]:
# model_path = "hdfs://namenode:9000/models/kmeans_model"
# model.save(model_path)
# print(f"Modèle K-Means sauvegardé dans HDFS : {model_path}")

In [ ]:
from pyspark.ml.clustering import KMeansModel

# Charger le modèle K-Means
loaded_model = KMeansModel.load(model_path)
print("Modèle K-Means chargé avec succès.")


In [48]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, col, when
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml.clustering import KMeansModel

# Initialisation de la session Spark
spark = SparkSession.builder \
    .appName("Imputation Hiérarchique et Clustering") \
    .getOrCreate()

# Charger les données d'entraînement
training_data = spark.sql("SELECT * FROM catalogue_co2_merge_processed")

# Charger les données de prédiction
new_data_df = spark.sql("SELECT * FROM immatriculations_data")

# Renommer `nom` en `modele` pour aligner les colonnes
new_data_df = new_data_df.withColumnRenamed("nom", "modele")

# Vérifier et ajouter les colonnes manquantes
columns_to_check = ["rejets_co2", "bonus_malus"]
for col_name in columns_to_check:
    if col_name not in new_data_df.columns:
        print(f"Ajout de la colonne manquante : {col_name}")
        new_data_df = new_data_df.withColumn(col_name, lit(None).cast("double"))

# Étape 1 : Calcul des moyennes
# Moyennes par modèle
avg_by_model = training_data.groupBy("modele").agg(
    F.mean("rejets_co2").alias("avg_rejets_co2_model"),
    F.mean("bonus_malus").alias("avg_bonus_malus_model")
)

# Moyennes par marque
avg_by_marque = training_data.groupBy("marque").agg(
    F.mean("rejets_co2").alias("avg_rejets_co2_marque"),
    F.mean("bonus_malus").alias("avg_bonus_malus_marque")
)

# Moyennes globales
avg_global = training_data.agg(
    F.mean("rejets_co2").alias("avg_rejets_co2_global"),
    F.mean("bonus_malus").alias("avg_bonus_malus_global")
).collect()[0]

# Ajouter les moyennes globales dans les données de prédiction
new_data_df = new_data_df.withColumn("avg_rejets_co2_global", lit(avg_global["avg_rejets_co2_global"]))
new_data_df = new_data_df.withColumn("avg_bonus_malus_global", lit(avg_global["avg_bonus_malus_global"]))

# Étape 2 : Joindre les moyennes
# Joindre les moyennes par modèle
new_data_df = new_data_df.join(avg_by_model, on="modele", how="left")

# Joindre les moyennes par marque
new_data_df = new_data_df.join(avg_by_marque, on="marque", how="left")

# Étape 3 : Remplir les colonnes manquantes hiérarchiquement
new_data_df = new_data_df.withColumn(
    "rejets_co2",
    when(col("rejets_co2").isNotNull(), col("rejets_co2"))  # Valeur existante
    .when(col("avg_rejets_co2_model").isNotNull(), col("avg_rejets_co2_model"))  # Par modèle
    .when(col("avg_rejets_co2_marque").isNotNull(), col("avg_rejets_co2_marque"))  # Par marque
    .otherwise(col("avg_rejets_co2_global"))  # Globale
)

new_data_df = new_data_df.withColumn(
    "bonus_malus",
    when(col("bonus_malus").isNotNull(), col("bonus_malus"))  # Valeur existante
    .when(col("avg_bonus_malus_model").isNotNull(), col("avg_bonus_malus_model"))  # Par modèle
    .when(col("avg_bonus_malus_marque").isNotNull(), col("avg_bonus_malus_marque"))  # Par marque
    .otherwise(col("avg_bonus_malus_global"))  # Globale
)

# Supprimer les colonnes intermédiaires de moyennes
new_data_df = new_data_df.drop(
    "avg_rejets_co2_model", "avg_bonus_malus_model",
    "avg_rejets_co2_marque", "avg_bonus_malus_marque",
    "avg_rejets_co2_global", "avg_bonus_malus_global"
)

print("\n--- Données imputées hiérarchiquement ---")
new_data_df.show(10, truncate=False)

# Étape 4 : Préparer les données pour le modèle
numerical_cols = ['prix', 'rejets_co2', 'bonus_malus', 'nbplaces', 'nbportes']

# Conversion explicite en type double
for col_name in numerical_cols:
    new_data_df = new_data_df.withColumn(col_name, col(col_name).cast("double"))

# Assemblage des colonnes en vecteur de caractéristiques
assembler = VectorAssembler(inputCols=numerical_cols, outputCol="features_raw")
assembled_df = assembler.transform(new_data_df)

# Normalisation des données
scaler = MinMaxScaler(inputCol="features_raw", outputCol="features")
scaler_model = scaler.fit(assembled_df)
normalized_df = scaler_model.transform(assembled_df)

# Étape 5 : Charger et appliquer le modèle K-Means
kmeans_model = KMeansModel.load("hdfs://namenode:9000/models/kmeans_model")
predicted_df = kmeans_model.transform(normalized_df)

# Ajouter des catégories basées sur les prédictions
predicted_df = predicted_df.withColumn(
    "categorie",
    when(col("prediction") == 1, "Berlines compactes économiques")
    .when(col("prediction") == 6, "Compactes premium")
    .when(col("prediction") == 5, "Monospaces familiaux 7 places")
    .when(col("prediction") == 4, "Berlines de luxe haut de gamme")
    .when(col("prediction") == 8, "Voitures sportives haut de gamme")
    .when(col("prediction") == 2, "Citadines économiques compactes")
    .when(col("prediction") == 0, "Berlines routières intermédiaires")
    .when(col("prediction") == 3, "Micro-citadines économiques")
    .when(col("prediction") == 7, "Monospaces compacts économiques")
    .otherwise("Non défini")
)

# Étape 6 : Résumé des catégories
print("\n--- Résumé des catégories par prédiction ---")
predicted_df.groupBy("prediction", "categorie").count().show()

# Étape 7 : Sauvegarder les résultats
output_path = "hdfs://namenode:9000/output/clustered_results_hierarchical.csv"
predicted_df.select(
    "marque", "modele", *numerical_cols, "prediction", "categorie"
).write.mode("overwrite").csv(output_path, header=True)

print(f"Résultats sauvegardés dans : {output_path}")



Ajout de la colonne manquante : rejets_co2
Ajout de la colonne manquante : bonus_malus

--- Données imputées hiérarchiquement ---
+----------+--------------+---------------+---------+-----------+--------+--------+-------+--------+-------+------------------+-----------------+
|marque    |modele        |immatriculation|puissance|longueur   |nbplaces|nbportes|couleur|occasion|prix   |rejets_co2        |bonus_malus      |
+----------+--------------+---------------+---------+-----------+--------+--------+-------+--------+-------+------------------+-----------------+
|marque    |nom           |immatriculation|NULL     |longueur   |NULL    |NULL    |couleur|NULL    |NULL   |62.1527272727273  |-2194.21320754717|
|Renault   |Laguna 2.0T   |3176 TS 67     |170      |longue     |5       |5       |blanc  |false   |27300.0|62.1527272727273  |-2194.21320754717|
|Volvo     |S80 T6        |3721 QS 49     |272      |tr�s longue|5       |5       |noir   |false   |50500.0|62.1527272727273  |-2194.2132075

Py4JJavaError: An error occurred while calling o4752.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 526.0 failed 1 times, most recent failure: Lost task 0.0 in stage 526.0 (TID 552) (74c859225edd executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`VectorAssembler$$Lambda$3681/0x00007f19dd1ec638`: (struct<prix:double,rejets_co2:double,bonus_malus:double,nbplaces:double,nbportes:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage5.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(Bu